# Take images for crosstalk determination

This notebook will find a bright star and take saturated images with the star in each amp\
See SITCOM-609 \
Craig Lage - 27Jan23

In [ ]:
import sys
import asyncio
import time
import astropy
import numpy as np
import logging 
import yaml
import os

from lsst.ts import salobj
from lsst.ts.idl.enums.Script import ScriptState
from lsst.ts.externalscripts.auxtel.latiss_acquire_and_take_sequence import LatissAcquireAndTakeSequence

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)
# if you want logging
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

# turn off logging for matplotlib
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

In [ ]:
print(f'Your UID is {os.getuid()}')
index=os.getuid()*10+np.random.randint(0,9)
print(f'The generated index is {index}')

In [ ]:
# We will use this script to acquire and center the star
script = LatissAcquireAndTakeSequence(index=index)  # this essentially calls the init method

In [ ]:
# make sure all remotes etc are running
await script.start_task

In [ ]:
# ATAOS must be on and corrections enabled, do as follows if required
# await script.atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

# Find a bright target

In [ ]:
target_name = await script.atcs.find_target(az=90.0,el=70,mag_limit=7.0)
print(target_name)

# Now acquire the object and the script will center it in the field

In [ ]:
# Needed to run the script a 2nd time 
script.set_state(ScriptState.UNCONFIGURED)

In [ ]:
configuration = yaml.safe_dump({"object_name": target_name,
                                "do_acquire": True,
                                "acq_filter" : 'SDSSr_65mm',
                                "acq_grating" : 'empty_1',
                                "do_take_sequence": False})
print(configuration)

In [ ]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

In [ ]:
# Run the script
group_id_data = script.cmd_setGroupId.DataType(
                groupId=astropy.time.Time.now().isot
            )
await script.do_setGroupId(group_id_data)

run_data = script.cmd_run.DataType()
await script.arun()

In [ ]:
# Not very well centered.  Centering
await script.atcs.offset_xy(x=-30, y=-20) # Move to upper left amp
await script.latiss.take_object(exptime=5.0, n=1, filter='SDSSr_65mm',grating='empty_1')

# Now take an image.
## Verify that the star is centered in the field and that the image is saturated.
## If it is not saturated, increase the exposure time until it is.

In [ ]:
expTime = 30.0
await script.latiss.take_object(exptime=expTime, n=1, filter='SDSSr_65mm',grating='empty_1')

In [ ]:
# Not well saturated
expTime = 45.0
await script.latiss.take_object(exptime=expTime, n=1, filter='SDSSr_65mm',grating='empty_1')

# Now center the star in the upper right amp and take another image
## Verify that the star is approximately centered in the upper right amp.

In [ ]:
await script.atcs.offset_xy(x=-175, y=-100) # Move to upper left amp
await script.latiss.take_object(exptime=expTime, n=1, filter='SDSSr_65mm',grating='empty_1')

# Now run the loop to walk the star through all 16 amps
## Verify that each of the 16 images has the star approximately centered in the 16 amps

In [ ]:
for i in range(2):
    if i == 0:
        direction = 1.0
    else:
        direction = -1.0
        await script.atcs.offset_xy(x=0, y=200) # Shift to bottom amps
        await script.latiss.take_object(exptime=expTime, n=1, filter='SDSSr_65mm',grating='empty_1')
    for j in range(7):
        await script.atcs.offset_xy(x=direction*50.9, y=0) # Shift to next amp
        await script.latiss.take_object(exptime=expTime, n=1, filter='SDSSr_65mm',grating='empty_1')

## If the 16 images are good, then we are done.

In [ ]:
await script.atcs.stop_tracking()